In [ ]:
#Reference https://rumn.medium.com/part-1-ultimate-guide-to-fine-tuning-in-pytorch-pre-trained-model-and-its-configuration-8990194b71e

In [ ]:
!pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split
import wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/CS6910-A2/CS6910-A2/train'
test_dir = '/content/drive/MyDrive/CS6910-A2/CS6910-A2/val'

#train_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'
#test_dir = '/kaggle/input/inaturalist/inaturalist_12K/val'
IMG_SIZE = (224,224)

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_dir, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mounted at /content/drive
cuda


In [ ]:
class pretrainedCNN():
  def __init__(self, trainset, valset, model, batch_size=32, freeze_percent = 1):
    # freeze earlier layers
    self.model = model
    if freeze_percent==1:
      for param in self.model.parameters():
        param.requires_grad = False
    else:
      total_layers = sum(1 for _ in model.parameters())
      num_layers_to_freeze = int(freeze_percent * total_layers)

      # Counter to track the number of frozen layers
      frozen_layers = 0

      # Freeze layers until reaching the desired percentage
      for param in self.model.parameters():
          param.requires_grad = False
          frozen_layers += 1
          if frozen_layers >= num_layers_to_freeze:
              break

    # Modify fully connected layer for fine-tuning
    num_classes = 10  # Number of classes in iNaturalist dataset
    num_ftrs = self.model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)
    for param in self.model.fc.parameters():
      param.requires_grad = True
    # DataLoader for the dataset
    self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
    self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)

  def train(self, epochs = 10, lr=0.001, weight_decay=0):
    self.model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(1,epochs+1):
      train_accuracy = 0
      count = 0
      self.model.train()
      for inputs, labels in self.dataloader_train:
          # forward, backward, and then weight update
          inputs = inputs.to(device)
          labels = labels.to(device)
          # Forward pass
          optimizer.zero_grad()
          y_pred = self.model(inputs)
          loss = criterion(y_pred, labels)
          train_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
          count += len(labels)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      train_accuracy /= count
      print("Epoch %d: model train accuracy %.2f%%" % (epoch, train_accuracy*100))
      wandb.log({ 'epoch': epoch, 'train_accuracy': train_accuracy * 100})
      self.model.eval()
      with torch.no_grad():
        val_accuracy = 0
        count = 0
        for inputs, labels in self.dataloader_val:
            inputs = inputs.to(device)
            labels = labels.to(device)
            y_pred = self.model(inputs)
            val_loss = criterion(y_pred, labels)
            val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
            count += len(labels)
        val_accuracy /= count
        print("Epoch %d: model validation accuracy %.2f%%" % (epoch, val_accuracy*100))
        wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})
        wandb.log({ 'epoch': epoch, 'validation_loss': val_loss * 100})


In [ ]:
pre_model = models.resnet50(pretrained=True)
cnn_model = pretrainedCNN(trainset, valset, pre_model, batch_size=64, freeze_percent = 0.9)
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(self.model.parameters(), lr=0.001, weight_decay=0)
cnn_model.train(10, 0.0001, 0.005)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 181MB/s]


In [ ]:
testset = datasets.ImageFolder(root=test_dir, transform=transform)
dataloader_test = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=True)
test_accuracy = 0
count = 0
loss_fn = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)
for inputs, labels in dataloader_test:
    inputs = inputs.to(device)
    labels = labels.to(device)
    print(type(inputs))
    print(inputs.shape)
    y_pred = cnn_model(inputs)
    test_loss = loss_fn(y_pred, labels)
    test_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
    count += len(labels)
test_accuracy /= count
print("Test accuracy %.2f%%" % (test_accuracy*100))